***stock Evaluation***
********************************************************************************
**This script aims to evaluate, based on the hystorical date of an stock, how much is convenient to reduce the occurence of the investment in the stock**

***Libraries Definition***

In [19]:
##Generic library for Array and Data-time format
import datetime as dt
import time
import glob
import math
import os
import numpy as np
import pandas as pd

##Generic library to create plots
import plotly.graph_objects as go
import plotly.subplots as sp

##Generic library to retrieve stock-Data
import yfinance as yf

GENERAL PURPOSE FUNCTIONS
=========================

In [18]:
##Return the DATA INFORMATIONS
def month_year():
    now = dt.datetime.now()
    return now.day, now.month, now.year


Here we have the function that retrieve the stock information based on the choosen ISIN

Generic Functions to retrieve Stock Data from Database
======================================================

In [17]:
##Return the yfinance.Ticker object that stores all the relevant stock informations
def get_stock_data(isin_string):
    stock_ticker = isin_string
    stock_data = yf.Ticker(stock_ticker)
    stock_data.info
    return stock_data


##Return the stock price value for the requested day
def get_stock_price(stock_data, date, timeout=600):
    start_time = time.time()
    while time.time() - start_time < timeout:
        if f"{date}" in stock_data.index:
            price_date = stock_data.loc[f"{date}"]
            return price_date
        else:
            # If no price data is found, increment the date by one day
            date = pd.to_datetime(date) + pd.Timedelta(days=1)
            date = date.strftime("%Y-%m-%d")
            time.sleep(1)  # wait for 1 second before trying again
    # If timeout is reached, raise an error
    raise ValueError(
        f"Failed to find price data for {stock_data.isin} on {date} after {timeout} seconds"
    )


##Return the hystorical data with date expressed as string --> Suitable for calculations
def get_stock_with_date_index_data(stock_data, start_date, end_date):
    try:
        hist_data = stock_data.history(start=start_date, end=end_date)
        hist_data = hist_data["Close"]
        hist_data.index = hist_data.index.strftime("%Y-%m-%d")
        return hist_data
    except Exception as e:
        print(f"Error: {e}")
        return None


Functions Related to Investment Strategies
=========================================

In [16]:
##Info About Investment
def get_info_investment(
    stock_data, initial_capital, start_date, end_date, purchase_frequency
):
    # Create a date range from start_date to end_date
    dates = pd.date_range(start_date, end_date, freq=purchase_frequency)
    # print(dates)
    # Initialize a list to store the purchase prices
    prices_list = []
    # Initialize a list to store the average costs
    average_cost_list = []
    # Initialize a list to store the extended version of average costs (between two consecutives buying moments)
    average_cost_list_extended = []
    # Initialize a list to store the amount of bought shares
    number_share_list = []
    # Initialize a list to store the amount of money spent
    total_purchase_amount_list = []
    # Initialize a list to store the date where I have bought
    dates_purchase_list = []
    # Initialize a DataFrame to store the daily gains respect to the average_cost_list
    daily_gain = 0
    daily_gains_df = pd.DataFrame(columns=["date", "daily_gain"])
    # Initialization of some internal values used for computation
    total_purchase_amount = 0
    number_share = 0
    days_diff = 0

    # Loop through each date in the date range
    for i, date in enumerate(dates):
        # Assume we're buying a fixed amount of the investment each time
        purchase_amount = initial_capital / len(dates)
        total_purchase_amount += purchase_amount
        # Collect the cost of the total investiment
        total_purchase_amount_list.append(total_purchase_amount)
        # print(f"total_purchase_amount {total_purchase_amount_list[-1]}", f"Date {date}")
        # Get the price of the investment on this date (e.g. from a database or API)
        # date must be a string
        date = date.strftime("%Y-%m-%d")
        price = get_stock_price(stock_data, date)
        # Collect date of each purchase
        dates_purchase_list.append(date)
        # Collect the cost per share for each purchase
        prices_list.append(price)
        # print(f"Price {prices_list[-1]}", f"Date {date}")
        # Calculate the number of shares
        number_share += purchase_amount / price
        number_share_list.append(number_share)
        # print(f"number_share {number_share_list[-1]}", f"Date {date}")
        # Calculate the average cost up to this point
        if i == 0:
            average_cost = price
            average_cost_list_extended.extend([average_cost] * 1)  # multiply by 1
        else:
            average_cost = total_purchase_amount / number_share
            days_diff = (pd.to_datetime(date) - dates[i - 1]).days
            average_cost_list_extended.extend([average_cost_list[-1]] * days_diff)
        # Add the average cost to the list of average costs
        average_cost_list.append(average_cost)
        # Calculate the final average cost
        final_average_cost = average_cost_list[-1]
    # In order to track the last purchase
    average_cost_list_extended.extend([average_cost_list[-1]] * days_diff)
    for i, (stock_price_element, average_cost_element) in enumerate(
        zip(stock_data, average_cost_list_extended)
    ):
        daily_gain = (
            (stock_price_element - average_cost_element) / average_cost_element
        ) * 100
        daily_gains_df.loc[len(daily_gains_df)] = [stock_data.index[i], daily_gain]
    return (
        average_cost_list,
        final_average_cost,
        daily_gains_df,
        number_share_list,
        total_purchase_amount_list,
        dates_purchase_list,
    )


# Find which is the best strategy of investment
def get_best_investment_strategy(results):
    best_strategy = None
    best_average_cost = float("inf")
    best_number_shares = 0

    for freq, result in results.items():
        average_cost = result["final_average_cost"]
        number_shares = result["number_share_list"][-1]
        last_date_purchase = result["dates_purchase_list"][-1]

        if average_cost < best_average_cost:
            best_strategy = freq
            best_average_cost = average_cost
            best_number_shares = number_shares
            best_last_date_purchase = last_date_purchase

    print(
        f"The winning strategy is {best_strategy} with an average cost of {best_average_cost:.2f} , {best_number_shares:.2f} shares and last purchase on {best_last_date_purchase}."
    )
    return (best_strategy, best_average_cost, best_number_shares)

GENERIC PLOT FUNCTIONS
===========================

In [15]:
##To be used in order to plot the stock behavior along two dates that you choose
def plot_stock_data(stock_data, start_date, end_date):
    try:
        hist_data = stock_data.history(start=start_date, end=end_date)
        fig = go.Figure(data=[go.Scatter(x=hist_data.index, y=hist_data["Close"])])
        fig.update_layout(
            title=f"stock <b>{stock_data.ticker}</b> Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
        )
        fig.show()
    except Exception as e:
        print(f"Error: {e}")


def create_plot(x, y, name_trace, name_graph):
    fig = go.Figure()
    for x_list, y_list, name in zip(x, y, name_trace):
        fig.add_trace(
            go.Scatter(
                x=x_list,
                y=y_list,
                mode="lines+markers",
                name=name,
            )
        )
    fig.update_layout(
        title=name_graph,
        xaxis_title="",
        yaxis_title="",
        legend_title="Legenda",
        hovermode="x",
    )
    fig.show()
    return fig


***MAIN CODE***
===========================

Hystorical Behavior of the Selected stock
===========================

In [20]:
### GENERIC INFORMATION ABOUT stock and its HYSTORICAL BEHAVIOR
# isin_string = "IE00B4L5Y983", ticker_string = "VWRA.L", ticker_string = "GME"
stock_under_test = input("Enter the stock ticker symbol: ")
# Time Informations
day, month, year = month_year()
start_date = input("Enter the start date (YYYY-MM-DD): ")
# end_date = input("Enter the end date (YYYY-MM-DD): ")
end_date = f"{year}-{month}-{day}"
stock_data_object = get_stock_data(stock_under_test)
stock_data_daily_values = plot_stock_data(stock_data_object, start_date, end_date)

Evaluation with DCA with Custom Purchase Frequency
===========================

User Input Data
===============

In [21]:
initial_capital = 10000
start_date_dca = input("Enter the start date for DCA Strategy (YYYY-MM-DD): ")
end_date_dca = input("Enter the end date for DCA Strategy (YYYY-MM-DD): ")
stock_data_dca_values = get_stock_with_date_index_data(
    stock_data_object, start_date_dca, end_date_dca
)
# end_date_dca = f"{year}-{month}-{day}"  ##Today
# end_date_dca = "2024-12-31"
purchase_frequencies = [
    "1M",
    "3M",
    "6M",
    "9M",
    "12M",
    "15M",
]  # range of purchase frequencies

Calculations
========================

In [22]:
results = {}

for freq in purchase_frequencies:
    (
        average_cost_list,
        final_average_cost,
        daily_gains_df,
        number_share_list,
        total_purchase_amount_list,
        dates_purchase_list,
    ) = get_info_investment(
        stock_data_dca_values,
        initial_capital,
        start_date_dca,
        end_date_dca,
        freq,
    )
    results[freq] = {
        "average_cost_list": average_cost_list,
        "final_average_cost": final_average_cost,
        "daily_gains_df": daily_gains_df,
        "number_share_list": number_share_list,
        "total_purchase_amount_list": total_purchase_amount_list,
        "dates_purchase_list": dates_purchase_list,
    }
    #


C:\Users\Davidde\AppData\Local\Temp\ipykernel_27620\3523432062.py:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Davidde\AppData\Local\Temp\ipykernel_27620\3523432062.py:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Davidde\AppData\Local\Temp\ipykernel_27620\3523432062.py:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Davidde\AppData\Local\Temp\ipykernel_27620\3523432062.py:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Davidde\AppData\Local\Temp\ipykernel_27620\3523432062.py:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Davidde\AppData\Local\Temp\ipykernel_27620\3523432062.py:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please

Plotting Section for DCA vs Sum Lump 
====================================

In [25]:
plots = [
    {
        "y": [stock_data_dca_values]
        + [result["average_cost_list"] for result in results.values()],
        "x": [stock_data_dca_values.index]
        + [result["dates_purchase_list"] for result in results.values()],
        "name_trace": ["Indice Tracciato"]
        + [f"Costo di Carico Medio {freq}" for freq in purchase_frequencies],
        "name_graph": "Grafico Acquisto con DCA",
    },
    {
        "y": [result["daily_gains_df"].daily_gain for result in results.values()],
        "x": [result["daily_gains_df"].date for result in results.values()],
        "name_trace": [f"Guadagno Medio con {freq}" for freq in purchase_frequencies],
        "name_graph": "Grafico Daily Gain con DCA",
    },
    {
        "y": [result["number_share_list"] for result in results.values()],
        "x": [result["dates_purchase_list"] for result in results.values()],
        "name_trace": [
            f"Numero di share per acquisto a {freq}" for freq in purchase_frequencies
        ],
        "name_graph": "Grafico Numero Share con DCA",
    },
    {
        "y": [result["total_purchase_amount_list"] for result in results.values()],
        "x": [result["dates_purchase_list"] for result in results.values()],
        "name_trace": [
            f"Variazione Capitale Investito per acquisto a {freq}"
            for freq in purchase_frequencies
        ],
        "name_graph": "Variazione Capitale Investito con DCA",
    },
]

for plot in plots:
    fig = create_plot(
        x=plot["x"],
        y=plot["y"],
        name_graph=plot["name_graph"],
        name_trace=plot["name_trace"],
    )


##Define which is the best strategy
(best_strategy, best_average_cost, best_number_shares) = get_best_investment_strategy(
    results
)

#### Strategia con 200MA

### Controllo la 200MA per capire se investire o meno
### Se il mio prezzo medio e' sopra la 200MA e la 200MA cresce allora non faccio niente, altrimenti quando average_cost <= 200MA, compro


The winning strategy is 12M with an average cost of 21.31 and 469.33 shares.
